In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

Read all the worksheets from the data excel file

In [2]:
orders_df = pd.read_excel('../data/Muesli Project raw data - group 3.xlsx', header=1, sheet_name=0)
campaign_data_df = pd.read_excel('../data/Muesli Project raw data - group 3.xlsx', sheet_name=1)
order_process_data_df = pd.read_excel('../data/Muesli Project raw data - group 3.xlsx', sheet_name=2)
interndata_study_df = pd.read_excel('../data/Muesli Project raw data - group 3.xlsx', sheet_name=3)

Check the headers of the DataFrames

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    print(df.columns)
    print('-' * 30)

Change column names to lower case and snake case.

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    print(df.columns)
    print('-' * 30)

Delete unneeded columns.

In [5]:
orders_df = orders_df.drop(
                    ['index', 'origin_channel', 'category', 'sub-category'],
                    axis=1)

campaign_data_df = campaign_data_df.drop(
                    ['customer_name'],
                    axis=1)

order_process_data_df = order_process_data_df.drop(
                    ['row_id', 'order_date'],
                    axis=1)

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    print(df.columns)
    print('-' * 30)

Check the number of rows and columns.

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    print(df.shape)

Check for duplicates.

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    print(df.duplicated().value_counts())
    print('---')
    print(df.shape)
    print('-' * 30)

Drop the duplicates.

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    df.drop_duplicates(inplace=True)
    print(df.duplicated().value_counts())
    print('---')

Print column-names to decide on which columns to merge.

In [ ]:
dataframes = [orders_df,
              campaign_data_df,
              order_process_data_df,
              interndata_study_df
              ]

for df in dataframes:
    print(df.columns)
    print('-' * 30)

---
---

We have to calculate and validate the company assumptions:
1. **Preparation**: Preparation process duration from order recieve to shipping (2 days): "Order Date" - "Ready to Ship Day": merge 'orders_df' and 'interndata_study_df'
2. **Ready to Ship to on Truck**: Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge 'order_process_data_df' and 'interndata_study_df'
3. **Ready to Ship to on Truck Express**: Efficacy of "Express Processing"; the order ought to be on the truck on the ready to ship day
4. **Order Delivery**: From "On Truck Scan" to "Arrival Scan Date": merge campaign_data_df and order_process_data_df
5. **Order Delivery 2**: From "Pickup Date" to "Arrival Scan Date": merge campaign_data_df and interndata_study_df
6. **Order Delivery total**: Get an image of the whole delivery process duration, from 'Order Date' to 'Arrival Scan Date': merge orders_df and campaign_data_df

---

### 1. **Preparation**: Preparation process duration from order recieve to shipping (2 days): "Order Date" - "Ready to Ship Day":
merge 'orders_df' and 'interndata_study_df'

In [11]:
preparation_df = pd.merge(orders_df, interndata_study_df,
                          on = 'order_id',
                          how = 'inner')

In [ ]:
preparation_df.columns

Delete unneeded columns.

In [13]:
preparation_df = preparation_df.drop(['ship_mode',
                                      'country/region',
                                      'city',
                                      'state',
                                      'postal_code',
                                      'region',
                                      'product_id',
                                      'sales',
                                      'quantity',
                                      'discount',
                                      'profit'], axis=1)

In [ ]:
preparation_df.sample(10)

Extract the weekday from the date column.

In [ ]:
preparation_df['weekday'] = preparation_df['order_date'].dt.day_name()

preparation_df.sample(10)

Convert the date columns to datetime format.

In [ ]:
preparation_df['order_date'] = pd.to_datetime(preparation_df['order_date'])
preparation_df['ready_to_ship_date'] = pd.to_datetime(preparation_df['ready_to_ship_date'])
preparation_df['pickup_date'] = pd.to_datetime(preparation_df['pickup_date'])

preparation_df.info()

Check the type of data in the columns.

In [ ]:
preparation_df.dtypes

Calculate the preparation duration for each order and print it in a new column.

In [ ]:
preparation_df['preparation_duration'] = (preparation_df['ready_to_ship_date']
                                            - preparation_df['order_date']
                                            ).dt.days

preparation_df.sample(10)

In [ ]:
preparation_df['preparation_duration'].unique()

Setting up seaborn and plotting the data as boxplots.

In [ ]:
sns.set_style('whitegrid')
custom_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.figure(figsize=(10, 6))
sns.boxplot(x = 'weekday',
            y = 'preparation_duration',
            data = preparation_df,
            order = custom_order,
            color = 'skyblue',
            medianprops = dict(color = 'red'))
plt.title('Distribition of Preparation Duration by Order Day')
plt.xlabel('Order Day of Week')
plt.ylabel('Preparation Duration (Days)')
plt.xticks(rotation = 45)
plt.tight_layout()
plt.show()

Calculate the mean Preparation Duration.

In [ ]:
mean_preparation_days = preparation_df['preparation_duration'].mean()

mean_preparation_days

**=> 4 days and ca. 4 hours**  
That's way more than the claimed 2 days preparation duration!

---

### 2. **Ready to Ship to on Truck**: Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express):
merge 'order_process_data_df' and 'interndata_study_df'

In [22]:
readytoship_truck_df = pd.merge(order_process_data_df, interndata_study_df,
                                on = 'order_id',
                                how = 'inner')

In [ ]:
readytoship_truck_df.columns

No columns to delete.

In [ ]:
readytoship_truck_df.sample(10)

Extract the weekday name from the date column.

In [ ]:
readytoship_truck_df['weekday'] = readytoship_truck_df['on_truck_scan_date'].dt.day_name()

readytoship_truck_df.sample(10)

No convertion to datetime needed (see below).

In [ ]:
readytoship_truck_df.info()

Calculate the duration of the process from 'Ready to Ship' to 'on Truck' and print it in a new column.

In [ ]:
readytoship_truck_df['ship_to_truck'] = (readytoship_truck_df['on_truck_scan_date']
                                            - readytoship_truck_df['ready_to_ship_date']
                                            ).dt.days

readytoship_truck_df.sample(10)

In [ ]:
readytoship_truck_df['ship_to_truck'].unique()

Filter for standard processing.

In [ ]:
standard_readytoship_truck_df = readytoship_truck_df[readytoship_truck_df['ship_mode'] == 'Standard Processing']

standard_readytoship_truck_df.sample(10)

Calculate the mean Preparation Duration for Standard Processing.

In [ ]:
mean_standard_days = standard_readytoship_truck_df['ship_to_truck'].mean()

mean_standard_days

**=> ca. 2 days**  
That's more than the claimed 1 day.

---

### 3. **Ready to Ship to on Truck Express**: Efficacy of "Express Processing"; the order ought to be on the truck on the ready to ship day

In [ ]:
express_readytoship_truck_df = readytoship_truck_df[readytoship_truck_df['ship_mode'] == 'Express']

express_readytoship_truck_df.sample(10)

Calculate the mean Preparation Duration for Express Processing.

In [ ]:
mean_express_days = express_readytoship_truck_df['ship_to_truck'].mean()

mean_express_days

**=> ca. 10 hours**  
Less than the claimed 1 day!

---

### 4. **Order Delivery**: From "On Truck Scan" to "Arrival Scan Date":
merge campaign_data_df and order_process_data_df

In [33]:
order_delivery_df = pd.merge(campaign_data_df, order_process_data_df,
                             on = 'order_id',
                             how = 'inner')

In [ ]:
order_delivery_df.columns

No columns to delete.

In [ ]:
order_delivery_df['delivery_time'] = (order_delivery_df['arrival_scan_date']
                                      - order_delivery_df['on_truck_scan_date']
                                      ).dt.days

order_delivery_df.sample(10)

In [ ]:
order_delivery_df['delivery_time'].unique()

Calculate the mean Order Delivery Duration.

In [ ]:
mean_order_delivery_days = order_delivery_df['delivery_time'].mean()

mean_order_delivery_days

**=> 4 days 14 hours**  
More than the claimed 3 days!

---

### 5. **Order Delivery 2**: From "Pickup Date" to "Arrival Scan Date":
merge campaign_data_df and interndata_study_df

In [38]:
order_delivery_df_2 = pd.merge(campaign_data_df, interndata_study_df,
                               on = 'order_id',
                               how = 'inner')

In [ ]:
order_delivery_df_2.columns

No columns to delete.

In [ ]:
order_delivery_df_2['delivery_time_2'] = (order_delivery_df_2['arrival_scan_date']
                                          - order_delivery_df_2['pickup_date']
                                          ).dt.days
order_delivery_df_2.sample(10)

In [ ]:
order_delivery_df_2['delivery_time_2'].unique()

Calculate the mean Order Delivery Duration.

In [ ]:
mean_order_delivery_days_2 = order_delivery_df_2['delivery_time_2'].mean()

mean_order_delivery_days_2

**=> 4 days 16 hours**  
More than the claimed 3 days!

---

### 6. **Order Delivery total**: Get an image of the whole delivery process duration, from 'Order Date' to 'Arrival Scan Date':
merge orders_df and campaign_data_df

In [43]:
order_delivery_total_df = pd.merge(orders_df, campaign_data_df,
                                on = 'order_id',
                                how = 'inner')

In [ ]:
order_delivery_total_df.columns

Delete unneeded columns.

In [45]:
order_delivery_total_df = order_delivery_total_df.drop(['ship_mode',
                                                        'country/region',
                                                        'city',
                                                        'state',
                                                        'postal_code',
                                                        'region',
                                                        'product_id',
                                                        'sales',
                                                        'quantity',
                                                        'discount',
                                                        'profit'],
                                                        axis=1)

In [ ]:
order_delivery_total_df.sample(10)

Calculate the Total Delivery Time.

In [ ]:
order_delivery_total_df['total_delivery_time'] = (order_delivery_total_df['arrival_scan_date']
                                                  - order_delivery_total_df['order_date']
                                                  ).dt.days
order_delivery_total_df.sample(10)

Calculate the mean Total Delivery Time.

In [ ]:
mean_delivery_total_days = order_delivery_total_df['total_delivery_time'].mean()

mean_delivery_total_days

**=> 10 days 21 hours**
Twice as long as the claimed 5 days!

---
---

Save all the dataframes to reuse them in other notebooks.

In [ ]:
%store orders_df
%store campaign_data_df
%store order_process_data_df
%store interndata_study_df

%store preparation_df
%store readytoship_truck_df
%store standard_readytoship_truck_df
%store express_readytoship_truck_df
%store order_delivery_df
%store order_delivery_df_2
%store order_delivery_total_df